Functions

In [801]:
import re
import numpy as np
import math
import time


indices = {}


def load(name):
    """
    Load the data

    params:
    name:   file directory and name
    """
    arr = []
    with open(name) as f:
        for i in f:
            x = [j.strip() for j in re.split(',|\[|\]', i)]
            xtmp = []
            xtmp.append(int(x[1]))
            xtmp.append(int(x[2]))
            xtmp2 = []
            xtmp2.append(int(x[4]))
            xtmp2.append(int(x[5]))
            arr.append(xtmp)
            arr.append(xtmp2)

    return arr


def dist(a, b):
    return np.sqrt(np.sum((a-b)**2))


def dist_2(x, y):
    x1 = x[0]
    x2 = x[1]
    y1 = y[0]
    y2 = y[1]
    return ((x1 - y1)**2 + (x2 - y2)**2)**0.5
    

def pairwise_distances(myArray):
    # Set negative values to zero
    myArray[myArray < 0] = 0
    d = np.sqrt(np.sum((myArray[:, np.newaxis, :] - myArray[np.newaxis, :, :])**2, axis=-1))
    return d


def buildSet(arr):
    my_set = set()
    my_keys = []
    c = 0
    for i in arr:
        x = (i[0], i[1])
        if x not in my_set:
            my_set.add((i[0], i[1]))
            my_keys.append(c)
            c += 1
    l = list(my_set)
    return l, my_keys


def create_map(lst):
    indices = {}
    for i, x in enumerate(lst):
        indices[x] = i
    return indices


def find_index(element):
    return indices[element]


def find_key(i):
    key = [k for k, v in indices.items() if v == i][0]
    return key

KD Tree

In [802]:
# Chat GPT helped me construct this
class Node:
    def __init__(self, data=None, split=None, left=None, right=None):
        self.data = data
        self.split = split
        self.left = left
        self.right = right

class KDTree:
    def __init__(self, points, graph):
        self.root = self.build_kdtree(points)
        self.graph = graph

    def get(self):
        return self.graph.get()
    
    def build_kdtree(self, points, depth=0):
        n = len(points)
        if n <= 0:
            return None
        
        k = len(points[0])
        axis = depth % k
        sorted_points = sorted(points, key=lambda point: point[axis])
        
        mid = n // 2
        return Node(
            data=sorted_points[mid],
            split=axis,
            left=self.build_kdtree(sorted_points[:mid], depth+1),
            right=self.build_kdtree(sorted_points[mid+1:], depth+1)
        )
    
    def search_knn(self, query, k=1):
        v = find_index(query)
        parent = self.graph.get_parent()
        v_parent = self.graph.find(parent, v)
        vertexes = self.graph.get_clusters()[v_parent]
        min = math.inf
        x = -1
        kn = -1
        # start_time = time.time()
        for i in vertexes:
            knn = []
            self.search_knn_helper(query, self.root, knn, k, vertexes)

            d = dist_2(knn[0], find_key(i))
            if(d < min):
                min = d
                x = i
                kn = knn[0]
        # end_time = time.time()
        # timer = round(end_time - start_time, 2)
        # print("1: ", timer)

        self.graph.add_edge(find_index(kn), x, min)
        self.graph.union()
        return kn
    
    # def search_knn(self, query, k=1):
    #     x, y = -1, -1
    #     v = find_index(query)
    #     while x == y:
    #         knn = []
    #         self.search_knn_helper(query, self.root, knn, k)
    #         u = find_index(knn[0])     
    #         parent = self.graph.get_parent()
    #         x = self.graph.find(parent, u)
    #         y = self.graph.find(parent, v)
    #         print(query, x, y)
    #     self.graph.add_edge(u, v, dist_2(query, knn[0]))
    #     return knn
    
    def closer_distrance(p1, p2, p):
        if p1 is None:
            return p2
        
        if p2 is None:
            return p1
        
        if dist_2(p, p1) < dist_2(p, p2):
            return p1

        return p2

    def search_knn_helper(self, query, node, knn, k, ex):
        if node is None:
            return
            
        # print(node.data)
        axis = node.split
        # print(query[axis], node.data[axis])
        if query[axis] < node.data[axis]:
            self.search_knn_helper(query, node.left, knn, k, ex)
            further_child = node.right
            closer_child = node.left
        else:
            self.search_knn_helper(query, node.right, knn, k, ex)
            further_child = node.left
            closer_child = node.right
            
        # print(knn)
        if find_index(node.data) not in ex and node.data not in knn:  # skip nodes whose point is the same as the query point or already in knn
            knn.append(node.data)
            knn.sort(key=lambda point: np.linalg.norm(np.array(point) - query))
            while len(knn) > k:
                knn.pop()
                
            # print(abs(query[axis] - node.data[axis]), np.linalg.norm(np.array(knn[-1]) - query))
            if further_child is not None and \
            abs(query[axis] - node.data[axis]) < np.linalg.norm(np.array(knn[-1]) - query):
                self.search_knn_helper(query, further_child, knn, k, ex)



Class

In [803]:
# Credit to: https://www.programiz.com/dsa/kruskal-algorithm
class Graph:
    def __init__(self, vertices, edge):
        self.V = vertices
        self.graph = edge
        self.parent = []
        self.rank = []
        for node in range(self.V):
            self.parent.append(node)
            self.rank.append(0)
        self.result = self.graph[:]
        self.graph = sorted(self.graph, key=lambda item: item[2])
        self.clusters = {}
        for k in self.result:
            u = k[0]
            v = k[1]
            x = self.find(self.parent, u)
            y = self.find(self.parent, v)
            self.apply_union(self.parent, self.rank, x, y)
        
        for i in range(self.V):
            parent = self.find(self.parent, i)
            if parent in self.clusters:
                self.clusters[parent].append(i)
            else:
                self.clusters[parent] = [i]
    
    def union(self):
        self.result = self.graph[:]
        u = self.result[-1][0]
        v = self.result[-1][1]
        x = self.find(self.parent, u)
        y = self.find(self.parent, v)
        self.apply_union(self.parent, self.rank, x, y)
        
        self.clusters = {}
        for i in range(self.V):
            parent = self.find(self.parent, i)
            if parent in self.clusters:
                self.clusters[parent].append(i)
            else:
                self.clusters[parent] = [i]

    def add_edge(self, u, v, w):
        self.graph.append([u, v, w])

    def get_parent(self):
        return self.parent
    
    def get_rank(self):
        return self.rank

    def get(self):
        return self.graph, self.V

    def find(self, parent, i):
        if parent[i] == i:
            return i
        parent[i] = self.find(parent, parent[i])
        return parent[i]

    def apply_union(self, parent, rank, x, y):
        xroot = self.find(parent, x)
        yroot = self.find(parent, y)
        if rank[xroot] < rank[yroot]:
            parent[xroot] = yroot
        elif rank[xroot] > rank[yroot]:
            parent[yroot] = xroot
        else:
            parent[yroot] = xroot
            rank[xroot] += 1

    def get_clusters(self):
        return self.clusters

Load

In [804]:
arr = load("./graf.txt")

Keys

In [805]:
myList, myKeys = buildSet(arr)
num_vertices = len(myList)
num_edges = int((num_vertices *  (num_vertices - 1)) / 2)

Indices

In [806]:
indices = create_map(myList)
indices

{(97158, 12327): 0,
 (97337, 12329): 1,
 (58892, 18643): 2,
 (69850, 24779): 3,
 (82285, 85635): 4,
 (7422, 91121): 5,
 (38459, 92363): 6,
 (69907, 24761): 7,
 (87727, 29436): 8,
 (34232, 592): 9,
 (81808, 27094): 10,
 (31549, 8857): 11,
 (89531, 2055): 12,
 (71303, 38165): 13,
 (41340, 67346): 14,
 (6683, 1547): 15,
 (36903, 83107): 16,
 (27913, 36452): 17,
 (93745, 48975): 18,
 (47306, 26911): 19,
 (81055, 27759): 20,
 (28091, 93613): 21,
 (73487, 64898): 22,
 (64695, 48258): 23,
 (73922, 64826): 24,
 (28819, 93534): 25,
 (36402, 83764): 26,
 (79302, 96203): 27,
 (18918, 95482): 28,
 (54763, 10734): 29,
 (32819, 32631): 30,
 (35996, 94861): 31,
 (81403, 27166): 32,
 (2536, 53957): 33,
 (57933, 87712): 34,
 (79925, 96596): 35,
 (85770, 53683): 36,
 (20084, 40423): 37,
 (18008, 95934): 38,
 (42180, 62196): 39,
 (31683, 8914): 40,
 (89940, 2088): 41,
 (85791, 53742): 42,
 (31347, 8470): 43,
 (77417, 6087): 44,
 (92185, 82014): 45,
 (57890, 87786): 46,
 (95703, 42800): 47,
 (77611, 6892)

Initial Edges

In [807]:
initial_edges = []
for i in range(0, len(arr), 2):
    initial_edges.append([find_index((arr[i][0], arr[i][1])), find_index((arr[i + 1][0], arr[i + 1][1])), dist_2(arr[i], arr[i + 1])])


Make Graph

In [808]:
g = Graph(num_vertices, initial_edges)
# g.get_parent()
# g.get()

Tree

In [809]:
kd = KDTree(myList, g)
i = 0
while len(g.get_clusters()) > 1:
    kd.search_knn(myList[i])
    # print(g.get())
    i += 1

2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.02
2:  0.01
2:  0.02
2:  0.02
2:  0.01
2:  0.03
2:  0.02
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.02
2:  0.01
2:  0.02
2:  0.02
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.02
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.02
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.02
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.01
2:  0.02
2:  0.01
2:  0.01
2:  0.01
2:  0.02
2:  0.01
2:  0.01
2:  0.01
2:  0.02
2:  0.01
2:  0.01
2:  0.01
2

KeyboardInterrupt: 

In [ ]:
g.get_parent()
g.get_clusters()

{0: [0, 3, 4, 6, 9, 11, 12],
 1: [1, 10],
 15: [2, 15],
 8: [5, 8, 14],
 16: [7, 16],
 13: [13, 17]}

Find new edges

In [ ]:
# while 4:
#     for i in myList:
#         kd.search_knn(i)

#     # check connectivity
#     visited = [False] * num_vertices
#     g.dfs(0, visited)

#     if all(visited):
#         break

# for i in myList:
#    kd.search_knn(i)

# g.get()


Algorithm

In [ ]:
# g.kruskal_algo(initial_edges)